In [5]:
import numpy as np
import pandas as pd
import time

            

Rember that live_stream need to run with following prerequisite. 
1. this jupyter notebook need to run on the computer that plays the league of legend
2. use gcloud init to connect to GCP so it can upload data to the bucket

In [6]:
def devide_team(player):   #this function is used to divide players into team
    team1name=[]    #create two list for team 1 and team2
    team2name=[]
    for i in range(len(player)):    #read through the player slist returned by the curl command
        if 'summonerName' in player[i]:        
            text=player[i]
            text=text.replace('\t\t"summonerName": "','')
            name=text.replace('",','')     #clean the line and only save the player name
            j=i
            while True:
                text=player[j]           #see which team is the player in
                if '"team": "ORDER"' in text:  #append player name to the team
                    team1name.append(name)
                    break
                if '"team": "CHAOS"' in text:
                    team2name.append(name)
                    break
                j+=1
    return team1name,team2name

In [7]:
def subtract_list(list1,list2):     #subtract two list by turning into np array and convert back
    array1 = np.array(list1)
    array2 = np.array(list2)
    subtracted_array = np.subtract(array1, array2)
    subtracted = list(subtracted_array)
    return subtracted


In [8]:
#Turret_T1:blue side
#Turret_T2:red side
#'\t\t\t"EventName": "TurretKilled",',
#'\t\t\t"EventName": "DragonKill",',
#'\t\t\t"EventName": "InhibKilled",',
# '\t\t\t"EventName": "HeraldKill",',
#'\t\t\t"EventName": "BaronKill",',

#this get_real_time is used to get the data up to this minute, we need to put the slist as input 
def get_real_time(team1,team2,data,bluedragon,reddragon,bluebaron,redbaron,blueherald,redherald,blueturrent,redturrent,blueinhib,redinhib,bluekill,redkill):
    bd=0   #these varaible are basically  counts
    rd=0
    bb=0
    rb=0
    bh=0
    rh=0
    bt=0
    rt=0
    bi=0
    ri=0
    bk=0
    rk=0
    for i in range(len(data)):
        #this one is turrent kill
        if 'TurretKilled' in data[i]:     #these blocks of "if" function read what each event is and only count features 
            text=data[i+3]                # that we want
            if 'T1' in text:
                rt+=1
            if 'T2' in text:
                bt+=1
            
            
            
        if 'DragonKill' in data[i]: #dragon kill
            text=data[i+2]
            text=text.replace('\t\t\t"KillerName": "','')
            name=text.replace('",','')
            if name in team1:
                bd+=1
            else:
                rd+=1
            
       
        if 'InhibKilled' in data[i]: #inhibitor kill
            
            text=data[i+2]
            if 'T1' in text:
                ri+=1
            if 'T2' in text:
                bi+=1
            
            
        if 'HeraldKill' in data[i]:  #herald kills
            text=data[i+2]
            text=text.replace('\t\t\t"KillerName": "','')
            name=text.replace('",','')
            if name in team1:
                bh+=1
            else:
                rh+=1
            
            
        if 'BaronKill' in data[i]: #baron kills
            text=data[i+2]
            text=text.replace('\t\t\t"KillerName": "','')
            name=text.replace('",','')
            if name in team1:
                bb+=1
            else:
                rb+=1
        if 'ChampionKill' in data[i]:  #champion kills
            text=data[i+2]
            name=text.replace('\t\t\t"KillerName": "','')
            name=name.replace('",','')
            #print(name)
            #print("team1",team1)
            #print('team2',team2)
            #print(rk)
            #print(bk)
            if name in team1:
                bk+=1
            if name in team2:
                rk+=1
    
    bluedragon.append(bd)  #append these counts to the list so we can return the total counts of features up to this minute
    reddragon.append(rd)
    bluebaron.append(bb)
    redbaron.append(rb)
    blueherald.append(bh)
    redherald.append(rh)
    blueturrent.append(bt)
    redturrent.append(rt)
    blueinhib.append(bi)
    redinhib.append(ri)
    bluekill.append(bk)
    redkill.append(rk)
    return bluedragon,reddragon,bluebaron,redbaron,blueherald,redherald,blueturrent,redturrent,blueinhib,redinhib,bluekill,redkill
    

In [11]:
def real_time():   #this real_time function is like the main function that upload the data each minute to the GCP bucket
    #dont forget to turn it off
    GCPbucket='gs://dataproc-staging-us-central1-419343931639-hthrtj25/leaguedata/'  #put the link to the GCP bucket 
    bluedragon=[] #create list for our features
    reddragon=[]
    bluebaron=[]
    redbaron=[]
    blueherald=[]
    redherald=[]
    blueturrent=[]
    redturrent=[]
    blueinhib=[]
    redinhib=[]
    bluekill=[]
    redkill=[]
    golddiff=[]
    winner=0
    
    players=!curl --insecure https://127.0.0.1:2999/liveclientdata/playerlist   #get the player info
    team1,team2=devide_team(players)  #divide team for the players
    while True:  #use while loop and time.sleep to get data every minute
        data=!curl --insecure https://127.0.0.1:2999/liveclientdata/eventdata  #get the event data
        bluedragon,reddragon,bluebaron,redbaron,blueherald,redherald,blueturrent,redturrent,blueinhib,redinhib,bluekill,redkill=get_real_time(team1,team2,data,bluedragon,reddragon,bluebaron,redbaron,blueherald,redherald,blueturrent,redturrent,blueinhib,redinhib,bluekill,redkill)
        golddiff.append(0) #dummy variable, because we want to keep the dataframe within the same format of our data that uses the match history to get
    
        df=pd.DataFrame([[golddiff,subtract_list(bluedragon,reddragon),subtract_list(bluebaron,redbaron),subtract_list(blueherald,redherald),subtract_list(blueturrent,redturrent),subtract_list(blueinhib,redinhib),subtract_list(bluekill,redkill),winner]],columns=['golddiff','dragondiff','barondiff','heralddiff','towerdiff','inhibitordiff','killdiff','winner'])
        df.to_csv('text.csv', index = False)  #create data frame and turn into csv
        print(df)  
        #!gsutil cp 'text.csv' 'gs://dataproc-staging-us-central1-419343931639-hthrtj25/leaguedata/'  #upload to gcp bucket
        !gsutil cp 'text.csv' GCPbucket #upload to the GCP bucket
        #!gsutil cp 'text.csv' 'gs://6893gs_data/leaguedata/'
        time.sleep(60)
    return
    

In [12]:
real_time() #remember to stop it when finished 

  golddiff dragondiff barondiff heralddiff towerdiff inhibitordiff killdiff  \
0      [0]        [0]       [0]        [0]       [0]           [0]      [0]   

   winner  
0       0  
Copying file://text.csv...
/ [1 files][  111.0 B/  111.0 B]                                                
Operation completed over 1 objects/111.0 B.                                      


KeyboardInterrupt: 